In [3]:
import pygame
import numpy as np
# Initialize pygame
pygame.init()

# Constants
WIDTH, HEIGHT = 800, 600
BACKGROUND_COLOR = (0, 0, 0)
CIRCLE_COLOR = (255, 255, 255)
CIRCLE_RADIUS = 20

# Initialize screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Interactive Circles")


# Constants
WIDTH, HEIGHT = 800, 600
BACKGROUND_COLOR = (0, 0, 0)
MAX_INTENSITY = 255  # For grayscale visualization

# Initialize screen
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Gaussian Beam Intensity")

# Beam parameters
k = 2 * np.pi / 500  # Example wave number for wavelength = 500 nm
beams = []  # List of Gaussian beams
selected_beam = None

# Create coordinate grid
x_vals = np.linspace(0, WIDTH, WIDTH)
y_vals = np.linspace(0, HEIGHT, HEIGHT)
x, y = np.meshgrid(x_vals, y_vals)

def gaussian_electric_field(w_0, z, k, x0, y0):
    r2 = (x - x0) ** 2 + (y - y0) ** 2
    w_z = w_0 * np.sqrt(1 + (z / (np.pi * w_0**2 / 500))**2)
    amplitude = (w_0 / w_z) * np.exp(-r2 / w_z**2)
    return amplitude

def compute_intensity():
    total_intensity = np.zeros((HEIGHT, WIDTH), dtype=np.float64)
    for beam in beams:
        w_0, z, x0, y0 = beam
        E = gaussian_electric_field(w_0, z, k, x0, y0)
        total_intensity += np.abs(E) ** 2
    if np.max(total_intensity) > 0:
        total_intensity = (total_intensity / np.max(total_intensity) * MAX_INTENSITY).astype(np.uint8)
    return np.dstack([total_intensity] * 3)  # Convert to RGB

circles = []  # List to store circle positions
selected_circle = None  # Track selected circle

running = True
while running:
    screen.fill(BACKGROUND_COLOR)  # Clear screen
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_a:
                circles.append([WIDTH // 2, HEIGHT // 2])  # Add new circle at center
            elif event.key == pygame.K_s and circles:
                circles.pop()  # Remove last circle
            elif event.key == pygame.K_PLUS or event.key == pygame.K_EQUALS:
                CIRCLE_RADIUS = min(100, CIRCLE_RADIUS + 5)  # Increase circle size
            elif event.key == pygame.K_MINUS:
                CIRCLE_RADIUS = max(5, CIRCLE_RADIUS - 5)  # Decrease circle size
        elif event.type == pygame.MOUSEBUTTONDOWN:
            mx, my = event.pos
            for i, (x, y) in enumerate(circles):
                if (x - mx) ** 2 + (y - my) ** 2 < CIRCLE_RADIUS ** 2:
                    selected_circle = i
                    break
        elif event.type == pygame.MOUSEBUTTONUP:
            selected_circle = None
        elif event.type == pygame.MOUSEMOTION and selected_circle is not None:
            circles[selected_circle] = list(event.pos)
    
    # Draw circles
    for x, y in circles:
        pygame.draw.circle(screen, CIRCLE_COLOR, (x, y), CIRCLE_RADIUS)
    
    pygame.display.flip()

pygame.quit()
